In [ ]:
"""
Prophet Time Series Project (Updated for Cultus Submission Requirements)
---------------------------------------------------------------------
This script focuses exclusively on the Facebook/Meta Prophet forecasting model and
implements a rigorous hyperparameter optimization and validation workflow required
by the project rubric. It produces the following deliverables required for submission:

- A realistic synthetic daily dataset with seasonality, holidays, changepoints, noise.
- A systematic hyperparameter tuning pipeline for Prophet (grid search and optional
  Bayesian search via Optuna if available) using rolling-origin cross-validation.
- A baseline comparison using Holt-Winters Exponential Smoothing and an optional
  SARIMA baseline (if pmdarima is available).
- A text-based report (Markdown) that details the hyperparameter search space,
  the validation strategy, and a quantitative comparison of Prophet vs baselines.
- All results and intermediate CSVs saved into an `output/` folder for submission.

Important: This file intentionally **does not** include any Neural ODEs, LSTMs,
or unrelated models. It focuses on Prophet and the classical baselines required.

Usage:
    python prophet_time_series_project.py --generate-data
    python prophet_time_series_project.py --run-all

Requirements (pip install):
    prophet numpy pandas scikit-learn statsmodels matplotlib optuna pmdarima

Note on packages:
- `prophet` (or `fbprophet` depending on your environment) is required.
- `optuna` is optional but recommended for faster Bayesian hyperparameter search.
- `pmdarima` is optional for SARIMA baseline.

"""

import os
import argparse
import json
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple
import warnings

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Model imports
try:
    from prophet import Prophet
except Exception:
    Prophet = None

try:
    from statsmodels.tsa.holtwinters import ExponentialSmoothing
except Exception:
    ExponentialSmoothing = None

try:
    import optuna
except Exception:
    optuna = None

try:
    import pmdarima as pm
except Exception:
    pm = None

# ----------------------------- Metrics -------------------------------------

def rmse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return mean_squared_error(y_true, y_pred, squared=False)


def mae(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return mean_absolute_error(y_true, y_pred)


def mase(y_true: np.ndarray, y_pred: np.ndarray, y_train: np.ndarray, seasonal_period: int = 1) -> float:
    """Mean Absolute Scaled Error (MASE).
    Uses seasonal naive with given seasonal_period as the denominator if seasonal_period>1,
    otherwise uses naive lag-1.
    """
    n = len(y_train)
    if n < 2:
        return np.inf
    if seasonal_period > 1 and n > seasonal_period:
        denom = np.mean(np.abs(y_train[seasonal_period:] - y_train[:-seasonal_period]))
    else:
        denom = np.mean(np.abs(y_train[1:] - y_train[:-1]))
    num = np.mean(np.abs(y_true - y_pred))
    return num / denom if denom != 0 else np.inf

# ------------------------- Data generation ---------------------------------

def generate_synthetic_daily_series(start_date: str = '2018-01-01', periods: int = 3 * 365, seed: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Generate a synthetic daily time series with:
    - piecewise linear trend with changepoints
    - yearly seasonality
    - weekly seasonality (weekday effect)
    - holiday spikes
    - noise and outliers

    Returns (df, holidays_df) where df has ['ds','y'] and holidays_df has ['ds','holiday']
    """
    np.random.seed(seed)
    dates = pd.date_range(start=start_date, periods=periods, freq='D')
    t = np.arange(periods)

    # Base linear trend
    trend = 0.02 * t
    # Introduce changepoints
    cp1 = int(periods * 0.28)
    cp2 = int(periods * 0.6)
    trend[cp1:] += 0.12 * (t[cp1:] - cp1)
    trend[cp2:] -= 0.09 * (t[cp2:] - cp2)

    # Yearly seasonality
    yearly = 8.0 * np.sin(2 * np.pi * t / 365.25)

    # Weekly effect: weekend bump
    weekday = np.array([0.0 if d.weekday() < 5 else 4.0 for d in dates])

    # Holidays: select some dates and apply multi-day window effect
    rng = np.random.default_rng(seed)
    num_holidays = max(6, int(periods / 180))
    holiday_indices = rng.choice(periods, size=num_holidays, replace=False)
    holiday_effect = np.zeros(periods)
    for hi in holiday_indices:
        start = max(0, hi - 1)
        end = min(periods, hi + 2)
        holiday_effect[start:end] += rng.normal(15, 4)

    # Noise and outliers
    noise = np.random.normal(0, 2.0, size=periods)
    outliers = np.zeros(periods)
    outlier_positions = rng.choice(periods, size=int(periods * 0.004), replace=False)
    outliers[outlier_positions] = rng.normal(30, 12, size=outlier_positions.shape[0])

    y = 45 + trend + yearly + weekday + holiday_effect + noise + outliers

    df = pd.DataFrame({'ds': dates, 'y': y})
    holidays = pd.DataFrame({'ds': dates[holiday_indices], 'holiday': ['synthetic_holiday'] * len(holiday_indices)})
    holidays = holidays.sort_values('ds').reset_index(drop=True)
    return df.reset_index(drop=True), holidays

# ------------------------- CV splits --------------------------------------

def rolling_origin_splits(n_obs: int, initial: int, horizon: int, step: int) -> List[Tuple[np.ndarray, np.ndarray]]:
    """Generate rolling-origin (expanding window) train/test index splits.

    Parameters
    ----------
    n_obs: total number of observations
    initial: initial training size (in observations)
    horizon: test horizon for each fold
    step: shift size for each fold
    """
    splits = []
    start = initial
    while start + horizon <= n_obs:
        train_idx = np.arange(0, start)
        test_idx = np.arange(start, start + horizon)
        splits.append((train_idx, test_idx))
        start += step
    return splits

# ----------------------- Prophet grid search -------------------------------

def grid_search_prophet(df: pd.DataFrame, holidays: pd.DataFrame, param_grid: Dict[str, List[Any]],
                        cv_initial: int = 365, cv_horizon: int = 90, cv_step: int = 90,
                        seasonal_period: int = 7, verbose: bool = True) -> Dict[str, Any]:
    """Perform grid search on Prophet hyperparameters using rolling-origin CV.

    Returns a dict with 'best_params' and 'results_df' (DataFrame of results).
    """
    if Prophet is None:
        raise ImportError("Prophet package not found. Install with `pip install prophet`")

    import itertools
    keys = list(param_grid.keys())
    combos = list(itertools.product(*(param_grid[k] for k in keys)))
    results = []

    n = df.shape[0]
    splits = rolling_origin_splits(n, initial=cv_initial, horizon=cv_horizon, step=cv_step)
    if verbose:
        print(f"Running grid search with {len(combos)} combinations and {len(splits)} CV folds...")

    for combo in combos:
        params = dict(zip(keys, combo))
        fold_metrics = []
        for train_idx, test_idx in splits:
            train_df = df.iloc[train_idx].reset_index(drop=True)
            test_df = df.iloc[test_idx].reset_index(drop=True)

            m = Prophet(
                changepoint_prior_scale=params.get('changepoint_prior_scale', 0.05),
                seasonality_mode=params.get('seasonality_mode', 'additive'),
                weekly_seasonality=params.get('weekly_seasonality', True),
                yearly_seasonality=params.get('yearly_seasonality', True)
            )
            # Set seasonality prior via attribute where supported
            if 'seasonality_prior_scale' in params:
                try:
                    m.seasonality_prior_scale = params['seasonality_prior_scale']
                except Exception:
                    pass

            if holidays is not None and not holidays.empty:
                m.holidays = holidays

            # Fit and predict
            m.fit(train_df)
            future = test_df[['ds']].copy()
            forecast = m.predict(future)
            y_true = test_df['y'].values
            y_pred = forecast['yhat'].values

            fold_metrics.append({
                'rmse': rmse(y_true, y_pred),
                'mae': mae(y_true, y_pred),
                'mase': mase(y_true, y_pred, train_df['y'].values, seasonal_period=seasonal_period)
            })

        avg_rmse = np.mean([fm['rmse'] for fm in fold_metrics])
        avg_mae = np.mean([fm['mae'] for fm in fold_metrics])
        avg_mase = np.mean([fm['mase'] for fm in fold_metrics])

        result_row = {**params, 'rmse': avg_rmse, 'mae': avg_mae, 'mase': avg_mase}
        results.append(result_row)

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(['mase', 'rmse']).reset_index(drop=True)
    best = results_df.iloc[0].to_dict()
    best_params = {k: best[k] for k in keys}
    return {'best_params': best_params, 'results_df': results_df}

# ---------------------- Optuna Bayesian search (optional) -------------------

def bayes_search_prophet(df: pd.DataFrame, holidays: pd.DataFrame, n_trials: int = 40,
                         cv_initial: int = 365, cv_horizon: int = 90, cv_step: int = 90,
                         seasonal_period: int = 7) -> Dict[str, Any]:
    """Use Optuna to tune Prophet hyperparameters. Returns best_params and study object.
    Optuna must be installed; otherwise this function will raise an informative error.
    """
    if optuna is None:
        raise ImportError("Optuna is not installed. Install with `pip install optuna` or use grid_search_prophet instead.")
    if Prophet is None:
        raise ImportError("Prophet package not found. Install with `pip install prophet`")

    n = df.shape[0]
    splits = rolling_origin_splits(n, initial=cv_initial, horizon=cv_horizon, step=cv_step)

    def objective(trial):
        cps = trial.suggest_loguniform('changepoint_prior_scale', 0.0005, 0.5)
        sps = trial.suggest_categorical('seasonality_prior_scale', [0.1, 0.5, 1.0, 3.0, 5.0, 10.0])
        mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
        weekly = trial.suggest_categorical('weekly_seasonality', [True, False])
        yearly = trial.suggest_categorical('yearly_seasonality', [True, False])

        fold_mases = []
        for train_idx, test_idx in splits:
            train_df = df.iloc[train_idx].reset_index(drop=True)
            test_df = df.iloc[test_idx].reset_index(drop=True)

            m = Prophet(changepoint_prior_scale=cps, seasonality_mode=mode,
                        weekly_seasonality=weekly, yearly_seasonality=yearly)
            try:
                m.seasonality_prior_scale = sps
            except Exception:
                pass
            if holidays is not None and not holidays.empty:
                m.holidays = holidays
            m.fit(train_df)
            forecast = m.predict(test_df[['ds']])
            fold_mases.append(mase(test_df['y'].values, forecast['yhat'].values, train_df['y'].values, seasonal_period=seasonal_period))

        return float(np.mean(fold_mases))

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    best_params = study.best_params
    return {'best_params': best_params, 'study': study}

# ------------------------- Model fit & forecast ----------------------------

def fit_prophet_and_forecast(df_train: pd.DataFrame, df_future: pd.DataFrame, params: Dict[str, Any], holidays: pd.DataFrame = None) -> pd.DataFrame:
    if Prophet is None:
        raise ImportError("Prophet package not found. Install with `pip install prophet`")

    m = Prophet(
        changepoint_prior_scale=params.get('changepoint_prior_scale', 0.05),
        seasonality_mode=params.get('seasonality_mode', 'additive'),
        weekly_seasonality=params.get('weekly_seasonality', True),
        yearly_seasonality=params.get('yearly_seasonality', True)
    )
    if 'seasonality_prior_scale' in params:
        try:
            m.seasonality_prior_scale = params['seasonality_prior_scale']
        except Exception:
            pass
    if holidays is not None and not holidays.empty:
        m.holidays = holidays

    m.fit(df_train)
    forecast = m.predict(df_future)
    # Keep only relevant columns
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# ------------------------- Baseline models --------------------------------

def fit_holt_winters_and_forecast(train_series: pd.Series, forecast_index: pd.DatetimeIndex, seasonal_periods: int = 7) -> pd.DataFrame:
    if ExponentialSmoothing is None:
        raise ImportError("statsmodels ExponentialSmoothing not available. Install statsmodels.")
    model = ExponentialSmoothing(train_series, trend='add', seasonal='add', seasonal_periods=seasonal_periods)
    fitted = model.fit(optimized=True)
    yhat = fitted.forecast(len(forecast_index))
    return pd.DataFrame({'ds': forecast_index, 'yhat': yhat.values})


def fit_sarima_and_forecast(train_series: pd.Series, forecast_index: pd.DatetimeIndex, seasonal_period: int = 7) -> pd.DataFrame:
    """Optional SARIMA baseline using pmdarima auto_arima. Returns DataFrame with ds and yhat.
    """
    if pm is None:
        raise ImportError("pmdarima not available. Install with `pip install pmdarima` to enable SARIMA baseline.")
    model = pm.auto_arima(train_series, seasonal=True, m=seasonal_period, error_action='ignore', suppress_warnings=True)
    yhat = model.predict(n_periods=len(forecast_index))
    return pd.DataFrame({'ds': forecast_index, 'yhat': yhat})

# ------------------------- Reporting --------------------------------------

def save_report(output_dir: str, config: Dict[str, Any], tuning_results: pd.DataFrame, best_params: Dict[str, Any], prophet_eval: Dict[str, Any], hw_eval: Dict[str, Any], sarima_eval: Dict[str, Any] = None) -> str:
    os.makedirs(output_dir, exist_ok=True)
    report_path = os.path.join(output_dir, 'prophet_tuning_report.md')

    lines = []
    lines.append('# Prophet Hyperparameter Tuning Report')
    lines.append('')
    lines.append('## Configuration')
    lines.append('')
    lines.append('```json')
    lines.append(json.dumps(config, indent=2))
    lines.append('```')
    lines.append('')

    lines.append('## Hyperparameter Search Space')
    lines.append('')
    lines.append('The grid or search space used for Prophet hyperparameter tuning:')
    lines.append('')
    lines.append(tuning_results.head(50).to_markdown(index=False))
    lines.append('')

    lines.append('## Best Parameters')
    lines.append('')
    lines.append('```json')
    lines.append(json.dumps(best_params, indent=2))
    lines.append('```')
    lines.append('')

    lines.append('## Evaluation on Holdout')
    lines.append('')
    lines.append('### Prophet Metrics')
    lines.append('')
    lines.append(pd.DataFrame([prophet_eval]).to_markdown(index=False))
    lines.append('')
    lines.append('### Holt-Winters Metrics')
    lines.append('')
    lines.append(pd.DataFrame([hw_eval]).to_markdown(index=False))
    lines.append('')
    if sarima_eval is not None:
        lines.append('### SARIMA Metrics')
        lines.append('')
        lines.append(pd.DataFrame([sarima_eval]).to_markdown(index=False))
        lines.append('')

    lines.append('## Validation Strategy')
    lines.append('')
    lines.append('* Rolling-origin (expanding window) cross-validation on training data')
    lines.append('* Initial training window: {}'.format(config.get('cv_initial')))
    lines.append('* Forecast horizon per fold: {}'.format(config.get('cv_horizon')))
    lines.append('* Step between folds: {}'.format(config.get('cv_step')))
    lines.append('')

    lines.append('## Notes and Recommendations')
    lines.append('')
    lines.append('- The primary metric used for model selection is MASE (scale-free).')
    lines.append('- Secondary metrics: RMSE and MAE.')
    lines.append('- For production, consider using Optuna for faster Bayesian tuning and enabling parallelization.')

    with open(report_path, 'w') as f:
        f.write('
'.join(lines))

    return report_path

# ------------------------- Pipeline ---------------------------------------

@dataclass
class ProjectConfig:
    start_date: str = '2018-01-01'
    periods: int = 3 * 365
    seed: int = 42
    cv_initial: int = 365
    cv_horizon: int = 90
    cv_step: int = 90
    output_dir: str = 'output'
    holdout_days: int = 180
    seasonal_period: int = 7


def run_pipeline(config: ProjectConfig, use_bayesian: bool = False, bayes_trials: int = 40) -> Dict[str, Any]:
    os.makedirs(config.output_dir, exist_ok=True)

    # 1. Generate synthetic data
    df, holidays = generate_synthetic_daily_series(start_date=config.start_date, periods=config.periods, seed=config.seed)
    df.to_csv(os.path.join(config.output_dir, 'simulated_series.csv'), index=False)
    holidays.to_csv(os.path.join(config.output_dir, 'simulated_holidays.csv'), index=False)

    # 2. Split into training (for tuning) and holdout
    holdout_days = config.holdout_days
    train_df = df.iloc[:-holdout_days].reset_index(drop=True)
    holdout_df = df.iloc[-holdout_days:].reset_index(drop=True)

    # 3. Define hyperparameter grid for Prophet (clear, documented)
    param_grid = {
        'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1],
        'seasonality_prior_scale': [1.0, 5.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative'],
        'weekly_seasonality': [True, False],
        'yearly_seasonality': [True, False]
    }

    config_dict = {
        'cv_initial': config.cv_initial,
        'cv_horizon': config.cv_horizon,
        'cv_step': config.cv_step,
        'param_grid_keys': list(param_grid.keys())
    }

    # 4. Hyperparameter tuning (grid or Bayesian)
    if use_bayesian and optuna is not None:
        print('Running Bayesian optimization via Optuna...')
        bayes_res = bayes_search_prophet(train_df, holidays, n_trials=bayes_trials,
                                         cv_initial=config.cv_initial, cv_horizon=config.cv_horizon, cv_step=config.cv_step,
                                         seasonal_period=config.seasonal_period)
        best_params = bayes_res['best_params']
        # Optuna uses floats/ints; try to make compatible keys
        # Add default keys if missing
        for k in ['weekly_seasonality', 'yearly_seasonality', 'seasonality_mode']:
            if k not in best_params:
                best_params[k] = True
        tuning_results_df = pd.DataFrame([best_params])
    else:
        print('Running grid search over Prophet hyperparameters...')
        tune_res = grid_search_prophet(train_df, holidays, param_grid,
                                       cv_initial=config.cv_initial, cv_horizon=config.cv_horizon, cv_step=config.cv_step,
                                       seasonal_period=config.seasonal_period)
        best_params = tune_res['best_params']
        tuning_results_df = tune_res['results_df']

    tuning_results_df.to_csv(os.path.join(config.output_dir, 'prophet_tuning_results.csv'), index=False)

    # 5. Fit optimized Prophet on full training set and forecast holdout
    prophet_forecast = fit_prophet_and_forecast(train_df, holdout_df[['ds']].copy(), best_params, holidays=holidays)
    prophet_forecast.to_csv(os.path.join(config.output_dir, 'prophet_holdout_forecast.csv'), index=False)

    prophet_eval = {
        'rmse': rmse(holdout_df['y'].values, prophet_forecast['yhat'].values),
        'mae': mae(holdout_df['y'].values, prophet_forecast['yhat'].values),
        'mase': mase(holdout_df['y'].values, prophet_forecast['yhat'].values, train_df['y'].values, seasonal_period=config.seasonal_period)
    }
    pd.DataFrame([prophet_eval]).to_csv(os.path.join(config.output_dir, 'prophet_holdout_metrics.csv'), index=False)

    # 6. Baseline: Holt-Winters
    try:
        hw_forecast = fit_holt_winters_and_forecast(train_df['y'], holdout_df['ds'], seasonal_periods=config.seasonal_period)
        hw_forecast.to_csv(os.path.join(config.output_dir, 'hw_holdout_forecast.csv'), index=False)
        hw_eval = {
            'rmse': rmse(holdout_df['y'].values, hw_forecast['yhat'].values),
            'mae': mae(holdout_df['y'].values, hw_forecast['yhat'].values),
            'mase': mase(holdout_df['y'].values, hw_forecast['yhat'].values, train_df['y'].values, seasonal_period=config.seasonal_period)
        }
        pd.DataFrame([hw_eval]).to_csv(os.path.join(config.output_dir, 'hw_holdout_metrics.csv'), index=False)
    except Exception as e:
        hw_eval = None
        print('Warning: Holt-Winters baseline not run:', str(e))

    # Optional SARIMA baseline if pmdarima available
    sarima_eval = None
    if pm is not None:
        try:
            sarima_forecast = fit_sarima_and_forecast(train_df['y'], holdout_df['ds'], seasonal_period=config.seasonal_period)
            sarima_forecast.to_csv(os.path.join(config.output_dir, 'sarima_holdout_forecast.csv'), index=False)
            sarima_eval = {
                'rmse': rmse(holdout_df['y'].values, sarima_forecast['yhat'].values),
                'mae': mae(holdout_df['y'].values, sarima_forecast['yhat'].values),
                'mase': mase(holdout_df['y'].values, sarima_forecast['yhat'].values, train_df['y'].values, seasonal_period=config.seasonal_period)
            }
            pd.DataFrame([sarima_eval]).to_csv(os.path.join(config.output_dir, 'sarima_holdout_metrics.csv'), index=False)
        except Exception as e:
            print('Warning: SARIMA baseline failed:', str(e))

    # 7. Generate text-based report for submission
    report_path = save_report(config.output_dir, config_dict, tuning_results_df, best_params, prophet_eval, hw_eval or {}, sarima_eval)

    # 8. Optional plotting (non-blocking)
    try:
        plt.figure(figsize=(12, 6))
        plt.plot(pd.concat([train_df, holdout_df])['ds'], pd.concat([train_df, holdout_df])['y'], label='observed')
        plt.plot(prophet_forecast['ds'], prophet_forecast['yhat'], label='Prophet')
        if hw_eval is not None:
            plt.plot(hw_forecast['ds'], hw_forecast['yhat'], label='Holt-Winters')
        if sarima_eval is not None:
            plt.plot(sarima_forecast['ds'], sarima_forecast['yhat'], label='SARIMA')
        plt.legend()
        plt.title('Observed vs Forecasts (Holdout)')
        plt.xlabel('ds')
        plt.ylabel('y')
        plt.tight_layout()
        plt.savefig(os.path.join(config.output_dir, 'forecast_comparison.png'))
        plt.close()
    except Exception:
        pass

    summary = {
        'best_params': best_params,
        'prophet_eval': prophet_eval,
        'hw_eval': hw_eval,
 'sarima_eval': sarima_eval,
        'tuning_results_csv': os.path.join(config.output_dir, 'prophet_tuning_results.csv'),
        'prophet_forecast_csv': os.path.join(config.output_dir, 'prophet_holdout_forecast.csv'),
        'hw_forecast_csv': os.path.join(config.output_dir, 'hw_holdout_forecast.csv') if hw_eval is not None else None,
        'report_md': report_path,
        'dataset_csv': os.path.join(config.output_dir, 'simulated_series.csv')
    }
    return summary

# ------------------------- CLI --------------------------------------------

def main():
    parser = argparse.ArgumentParser(description='Prophet Time Series Project - Cultus Submission Ready')
    parser.add_argument('--generate-data', action='store_true', help='Only generate and save the synthetic dataset')
    parser.add_argument('--run-all', action='store_true', help='Run full pipeline including tuning and evaluation')
    parser.add_argument('--output-dir', type=str, default='output', help='Output directory')
    parser.add_argument('--bayes', action='store_true', help='Use Bayesian tuning (Optuna) instead of grid search')
    parser.add_argument('--trials', type=int, default=40, help='Number of Optuna trials when using Bayesian tuning')
    args = parser.parse_args()

    config = ProjectConfig(output_dir=args.output_dir)

    if args.generate_data:
        df, holidays = generate_synthetic_daily_series(start_date=config.start_date, periods=config.periods, seed=config.seed)
        os.makedirs(config.output_dir, exist_ok=True)
        df.to_csv(os.path.join(config.output_dir, 'simulated_series.csv'), index=False)
        holidays.to_csv(os.path.join(config.output_dir, 'simulated_holidays.csv'), index=False)
        print(f"Saved simulated dataset to {os.path.join(config.output_dir, 'simulated_series.csv')}")
        return

    if args.run_all:
        print('Running full pipeline... this may take time depending on tuning configuration.')
        summary = run_pipeline(config, use_bayesian=args.bayes, bayes_trials=args.trials)
        print('Pipeline complete. Summary:')
        for k, v in summary.items():
            print(f'  {k}: {v}')
        return

    parser.print_help()


if __name__ == '__main__':
    main()
